In [1]:
import numpy as np
from entsoe import EntsoePandasClient
import pandas as pd
from functools import reduce

### API initialization

In [2]:
f = open('api_token', 'r')
token = f.read()

In [3]:
client = EntsoePandasClient(api_key=token)

### Use API to scrap data about solar and wind generation

In [44]:
def entsoe_api_generation(start_time: str, end_time: str, country: str, plant: str):
    start = pd.Timestamp(start_time, tz='Europe/Brussels')
    end = pd.Timestamp(end_time, tz='Europe/Brussels')
    h_in_year = (end - start).days * 24
    try:
        df = client.query_generation(country, start=start,end=end, psr_type=plant)
    except:
        df = pd.DataFrame(0, index=np.arange(8760), columns=[plant])
    if len(df) > h_in_year:
        i = 0
        stop = len(df) - 1
        new_agg = []
        while i<stop:
            var = (df.iloc[i,0] + df.iloc[i+1,0] + df.iloc[i+2,0] + df.iloc[i+3,0])/4
            new_agg.append(var)
            i = i + 4
        df = pd.DataFrame({plant:new_agg})
    else:
        df = df.iloc[:,0]
        df.reset_index(drop=True)
    return df

In [45]:
def entsoe_api_dataframe(start_time: str, end_time: str, country: str):
    plant_type = ["B16", "B18", "B19"]
    plant_idx = ["i10", "i8", "i3"]
    df_solar = entsoe_api_generation(start_time, end_time, country, plant_type[0]).reset_index(drop=True)
    df_offshore = entsoe_api_generation(start_time, end_time, country, plant_type[1]).reset_index(drop=True)
    df_onshore = entsoe_api_generation(start_time, end_time, country, plant_type[2]).reset_index(drop=True)
    data_frames = [df_solar, df_offshore, df_onshore]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), data_frames)
    df_merged.columns = plant_idx
    return df_merged

In [46]:
def all_country(start_time = '20210101', end_time = '20220101'):
    df_de = entsoe_api_dataframe(start_time, end_time, "DE_LU").add_suffix('_de')
    df_france = entsoe_api_dataframe(start_time, end_time, "FR").add_suffix('_fr')
    df_be = entsoe_api_dataframe(start_time, end_time, "BE").reset_index(drop=True)
    df_nl = entsoe_api_dataframe(start_time, end_time, "NL")
    df_benelux = df_be + df_nl
    df_benelux = df_benelux.add_suffix("_bnl")
    df = df_de.join(df_france).join(df_benelux)
    df = df.fillna(0)
    return df

In [47]:
df = all_country()

<ipython-input-45-dea9633093bf>:8: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,


In [49]:
df = df.transpose()

In [50]:
df.to_excel("output_t.xlsx", sheet_name="Generation")

### Scrap load data

In [51]:
def entsoe_api_load(start_time: str, end_time: str, country: str):
    start = pd.Timestamp(start_time, tz='Europe/Brussels')
    end = pd.Timestamp(end_time, tz='Europe/Brussels')
    h_in_year = (end - start).days * 24
    try:
        df = client.query_load(country, start=start,end=end)
    except:
        df = pd.DataFrame(0, index=np.arange(8760), columns=[country])
    if len(df) > h_in_year:
        i = 0
        stop = len(df) - 1
        new_agg = []
        while i<stop:
            var = (df.iloc[i,0] + df.iloc[i+1,0] + df.iloc[i+2,0] + df.iloc[i+3,0])/4
            new_agg.append(var)
            i = i + 4
        df = pd.DataFrame({"Load":new_agg})
    else:
        df.columns = ["Load"]
        df.reset_index(drop=True)
    return df

In [52]:
def load_country(start_time = '20210101', end_time = '20220101'):
    df_de = entsoe_api_load(start_time, end_time, "DE").add_suffix('_de')
    df_france = entsoe_api_load(start_time, end_time, "FR").add_suffix('_fr').reset_index(drop=True)
    df_be = entsoe_api_load(start_time, end_time, "BE")
    df_nl = entsoe_api_load(start_time, end_time, "NL")
    df_lu = entsoe_api_load(start_time, end_time, "LU")
    df_benelux = df_be + df_nl + df_lu
    df_benelux = df_benelux.add_suffix("_bnl")
    df = df_de.join(df_france, how="outer").join(df_benelux)
    df = df.fillna(0)
    return df

In [53]:
df_load = load_country()

In [55]:
df_load.head()

,Load_de,Load_fr,Load_bnl
0,44586.00,65761.0,21102.50
1,42832.25,64139.0,20441.00
2,41111.00,62657.0,19541.25
3,40364.50,59481.0,18834.00
4,40318.00,57656.0,18477.75


In [39]:
start_time = '20210101'
end_time = '20220101'
df_france = entsoe_api_load(start_time, end_time, "FR")

In [40]:
df_france= df_france.reset_index(drop=True)

In [41]:
df_france

,Load
0,65761.0
1,64139.0
2,62657.0
3,59481.0
4,57656.0
...,...
8746,56693.0
8747,54155.0
8748,51945.0
8749,52907.0


In [42]:
df_load

,Load_de,Load_fr,Load_bnl
0,44586.00,65761.0,21102.50
1,42832.25,64139.0,20441.00
2,41111.00,62657.0,19541.25
3,40364.50,59481.0,18834.00
4,40318.00,57656.0,18477.75
...,...,...,...
8755,55785.75,0.0,22331.25
8756,51849.00,0.0,21203.00
8757,48752.00,0.0,20126.00
8758,47461.50,0.0,0.00


In [57]:
df_load['Load_bnl']=df_load['Load_bnl'].replace(0,df_load['Load_bnl'].median())
df_load['Load_fr']=df_load['Load_fr'].replace(0,df_load['Load_fr'].median())

In [59]:
df_load = df_load.transpose()

In [60]:
df_load.to_excel("load_output_t.xlsx", sheet_name="Load")

### NTC value scrap

In [35]:
def entsoe_api_ntc(start_time = '20210101', end_time = '20220101'):
    NEIGHBOURS = {
    'DE_LU': ['BE', 'FR', 'NL'],
    'FR': ['BE', 'DE_LU'],
    'BE': ['NL', 'FR', 'DE_LU'],
    'NL': ['BE', 'DE_LU'],
    }
    NTC_values = pd.DataFrame(columns=['DE_LU', 'FR', 'BE', 'NL'], index=['DE_LU', 'FR', 'BE', 'NL'])
    start = pd.Timestamp(start_time, tz='Europe/Brussels')
    end = pd.Timestamp(end_time, tz='Europe/Brussels')
    for key in NEIGHBOURS:
        for values in NEIGHBOURS[key]:
            max_value = max(client.query_crossborder_flows(key, values, start=start, end=end))
            NTC_values.loc[key, values] = max_value
    return NTC_values

In [36]:
NTC_values = entsoe_api_ntc()

In [40]:
NTC_values = NTC_values.fillna(0)

In [43]:
NTC_values.to_excel("NTC_values.xlsx", sheet_name="NTC")

## Price scrap

In [11]:
start_time = '20210101'
end_time = '20210201'
start = pd.Timestamp(start_time, tz='Europe/Brussels')
end = pd.Timestamp(end_time, tz='Europe/Brussels')
df = client.query_day_ahead_prices("FR", start=start,end=end)

In [17]:
def price_scrap_country(country: str, start_time: str, end_time: str):
    start = pd.Timestamp(start_time, tz='Europe/Brussels')
    end = pd.Timestamp(end_time, tz='Europe/Brussels')
    df = client.query_day_ahead_prices(country, start=start,end=end)
    return df[:-1]

In [44]:
def price_scrap(start_time = '20210101', end_time = '20220101'):
    country_list = ["DE_LU", "FR", "BE", "NL"]
    df_de = price_scrap_country("DE_LU", start_time, end_time).add_suffix('_de').reset_index(drop=True)
    df_fr = price_scrap_country("FR", start_time, end_time).add_suffix('_fr').reset_index(drop=True)
    df_be = price_scrap_country("BE", start_time, end_time)
    df_nl = price_scrap_country("NL", start_time, end_time)
    df_bnl = (df_be+df_nl)/2
    return [df_de, df_fr, df_bnl]

In [45]:
df = price_scrap()

In [47]:
df[2]=df[2].reset_index(drop=True)

In [49]:
with pd.ExcelWriter('output_price.xlsx') as writer:
    df[0].to_excel(writer, sheet_name='Germany')
    df[1].to_excel(writer, sheet_name='France')
    df[2].to_excel(writer, sheet_name='Benelux')